In [1]:
# imports
import pandas as pd
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# allows to output plots in the notebook
%matplotlib inline

In [2]:
# changing to correct directory
# DO NOT RUN A SECOND TIME (Restart & Run All is allowed)
os.chdir("..")

## Figuring out when is electricity the cheapest
 - Current dateTime format is CET (UTC+1).
 - Finding the right hours
    - This can be done most easily by grouping by hour and then checking which hours have the average lowest prices
    - Also a bit more advanced would be to convert it into percentages and then find it by percentages (100% being the daily highest and 0 being 0%)
    - Both approaches should give approx the same answer

In [7]:
# reading in files
nordpool = pd.read_csv(os.path.join("data", "processed", "nordpool_estonia.csv"))
#nordpool['DateTime'] = pd.to_datetime(nordpool.pop('Date')) + pd.to_timedelta(nordpool.pop('Time'))
nordpool.dropna(inplace=True)
nordpool.drop_duplicates(inplace=True)
nordpool[:24]

,Date,Time,elspot_price,consumption
0,2016-01-01,00:00:00,16.39,911.0
1,2016-01-01,01:00:00,16.04,882.0
2,2016-01-01,02:00:00,15.74,854.0
3,2016-01-01,03:00:00,15.57,828.0
4,2016-01-01,04:00:00,15.47,821.0
5,2016-01-01,05:00:00,15.75,823.0
6,2016-01-01,06:00:00,15.36,842.0
7,2016-01-01,07:00:00,15.37,864.0
8,2016-01-01,08:00:00,15.19,869.0
9,2016-01-01,09:00:00,15.39,910.0
